In [1]:
import torch
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download,

In [2]:
text_1 = "I understand equations, both the simple and quadratical."
text_2 = "What kind of equations do I understand?"

# Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)
indexed_tokens

[101,
 146,
 2437,
 11838,
 117,
 1241,
 1103,
 3014,
 1105,
 186,
 18413,
 21961,
 1348,
 119,
 102,
 1327,
 1912,
 1104,
 11838,
 1202,
 146,
 2437,
 136,
 102]

In [3]:
tokenizer.convert_ids_to_tokens([str(token) for token in indexed_tokens])

['[CLS]',
 'I',
 'understand',
 'equations',
 ',',
 'both',
 'the',
 'simple',
 'and',
 'q',
 '##uad',
 '##ratic',
 '##al',
 '.',
 '[SEP]',
 'What',
 'kind',
 'of',
 'equations',
 'do',
 'I',
 'understand',
 '?',
 '[SEP]']

In [4]:
tokenizer.decode(indexed_tokens)

'[CLS] I understand equations, both the simple and quadratical. [SEP] What kind of equations do I understand? [SEP]'

In [5]:
cls_token = 101
sep_token = 102

In [6]:
def get_segment_ids(indexed_tokens):
    segment_ids = []
    segment_id = 0
    for token in indexed_tokens:
        if token == sep_token:
            segment_id += 1
        segment_ids.append(segment_id)
    segment_ids[-1] -= 1  # Last [SEP] is ignored
    return torch.tensor([segment_ids]), torch.tensor([indexed_tokens])

In [7]:
segments_tensors, tokens_tensor = get_segment_ids(indexed_tokens)
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [8]:
tokenizer.mask_token

'[MASK]'

In [9]:
tokenizer.mask_token_id

103

In [10]:
indexed_tokens[masked_index] = tokenizer.mask_token_id
tokens_tensor = torch.tensor([indexed_tokens])
tokenizer.decode(indexed_tokens)

NameError: name 'masked_index' is not defined

In [11]:
masked_lm_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForMaskedLM', 'bert-base-cased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
masked_lm_model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [13]:
embedding_table = next(masked_lm_model.bert.embeddings.word_embeddings.parameters())
embedding_table

Parameter containing:
tensor([[-0.0005, -0.0416,  0.0131,  ..., -0.0039, -0.0335,  0.0150],
        [ 0.0169, -0.0311,  0.0042,  ..., -0.0147, -0.0356, -0.0036],
        [-0.0006, -0.0267,  0.0080,  ..., -0.0100, -0.0331, -0.0165],
        ...,
        [-0.0064,  0.0166, -0.0204,  ..., -0.0418, -0.0492,  0.0042],
        [-0.0048, -0.0027, -0.0290,  ..., -0.0512,  0.0045, -0.0118],
        [ 0.0313, -0.0297, -0.0230,  ..., -0.0145, -0.0525,  0.0284]],
       requires_grad=True)

In [14]:
embedding_table.shape

torch.Size([28996, 768])

In [15]:
with torch.no_grad():
    predictions = masked_lm_model(tokens_tensor, token_type_ids=segments_tensors)
predictions

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3227,  -7.1947,  -7.3926,  ...,  -6.0100,  -5.7544,  -6.1663],
         [ -7.8402,  -8.0616,  -7.8916,  ...,  -6.5215,  -6.3073,  -6.8117],
         [ -7.5290,  -7.7294,  -7.5514,  ...,  -5.7504,  -5.5850,  -4.2405],
         ...,
         [ -6.8125,  -6.9280,  -7.1593,  ...,  -6.6268,  -4.8445,  -3.3929],
         [-11.4017, -11.5903, -11.0830,  ..., -10.0579,  -8.2003, -10.3975],
         [-12.9638, -13.2358, -13.1546,  ..., -10.3690, -11.4416, -10.5666]]]), hidden_states=None, attentions=None)

In [16]:
predictions[0].shape

torch.Size([1, 24, 28996])

In [17]:
# Get the predicted token
predicted_index = torch.argmax(predictions[0][0], dim=1)[masked_index].item()
predicted_index

NameError: name 'masked_index' is not defined

In [ ]:
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token